# Naive Bayes Klassifikator
Als einführendes Beispiel wollen wir mit Hilfe des Naive-Bayes Klassifikators Obsorten, Äpfel und Birnen, andhand des Gewichts und Zuckergehalts klassifizieren.

Wir laden die Daten in ein Dataframe: Zuckergehalt und Gewicht von Äpfeln und Birnen:

In [ ]:
import pandas as pd

url="https://raw.githubusercontent.com/troescherw/datasets/master/obst.csv"
df = pd.read_csv(url, delimiter=";")
df

Das Feature *Obstsorte* ist nominal skaliert. Deshalb ersetzen wir die String-Wert durch Dummys (One-Hot-Encoding):

In [ ]:
df = pd.get_dummies(df, drop_first=True)
df

**0** steht nun für **Apfel**, **1** für **Birne**. Beim **Naive Bayes Klassifikator** ist es übrigens nicht zwingend erforderlich (wie bei OLS), eine Kategorie zu entfernen, hab´s hier aber dennoch mal durchgeführt. Wir können nun unser Modell erstellen.

In [ ]:
from sklearn.naive_bayes import GaussianNB

X = df[["Zuckergehalt", "Gewicht"]]
y = df.Obstsorte_Birne
model = GaussianNB().fit(X,y)

Mit Hilfe des Modells können wir nun zwei "unbekannte" Obststücke klassifiziert werden. Haben wir ein Stück Obst, das z.B. ein Zuckergehalt von 52,5g und ein Gewicht von 125g verfügt, fragen wir das Modell, ob es sich um einen Apfel oder eine Birne handelt:

In [ ]:
unbekanntesObst = pd.DataFrame({"Zuckergehalt": [12.5], "Gewicht":[125]})
model.predict(unbekanntesObst)

Das Modell gibt eine **0** zurück, damit handelt es sich um einen Apfel. Wir erstellen ein Scatterplot und fügen dort auch das unbekannte Stück Obst ein:

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(df[df.Obstsorte_Birne==0].Zuckergehalt, df[df.Obstsorte_Birne==0].Gewicht, c="r", label="Apfel")
plt.scatter(df[df.Obstsorte_Birne==1].Zuckergehalt, df[df.Obstsorte_Birne==1].Gewicht, c="g", label="Birne")
plt.scatter(unbekanntesObst.Zuckergehalt, unbekanntesObst.Gewicht, c="y", label="Unbekannt")
plt.xlabel("Zuckergehalt in g/100g")
plt.legend()
plt.plot()

## Beispiel: Naive-Bayes-Modell für die Vorhersage einer Diabetes-Erkrankung
Wir erstellen anhand des bereits verwendeten Datensatzes *Diabetes* voraus, ob ein Patient anhand der vorliegenden Daten an Diabetes erkranken wird. Um die Modellqualität zu prüfen führen wir eine k-Fold-Cross-Validation durch (mit k=10).

| Feature  | Bedeutung                                  |
|----------|--------------------------------------------|
| age      | age in years                               |
| sex      | sex (1 = male, 0 = female)                 |
| cp       | chest pain type                            |
| trestbps | resting blood pressure im mm Hg            |
| chol     | serum cholesteral in mg/dl                 |
| fbs      | fasting blood sugar > 120 mg/dl (1=true, 0=false)|
| thalach  | maximum heart rate achieved                |
| restecg  | resting electrocardiographic results       |
| exang    | exercise induced angina (0=no, 1=yes)      |
| oldpeak  | ST depression induced by exercise relative to rest|
| slope    | the slope of the peak exercise ST segment  |
| ca       | number of major vessels (0-3) colored by flouroscopy|
| thal     | 3=normal, 6=fixed defect, 7=reversable defect|
| target   | 1 or 0                                     |

Zuerst laden wir den Datensatz in ein Pandas DataFrame:

In [ ]:
import pandas as pd
url="https://raw.githubusercontent.com/troescherw/datasets/master/diabetes.csv"
df = pd.read_csv(url)
df.head()

Wir splitten in X und y auf.

In [ ]:
X = df.iloc[:,:12]
y = df.target

Wir teilen die Datensätze in 10 Folds auf.

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True)

Nun iterieren wir durch die Folds und trainieren das Modell jeweils mit den Daten in den Folds:

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

scores = [] # Leere Liste für Scores

for index_train, index_test in kf.split(X):
    X_train = X.iloc[index_train]
    X_test = X.iloc[index_test]
    y_train = y.iloc[index_train]
    y_test = y.iloc[index_test]
    model.fit(X_train, y_train)
    scores.append(model.score(X_test, y_test))

scores


In [ ]:
from statistics import mean
mean(scores)